### Word-Level 번역기
- 글자 단위가 아닌 단어 단위로 동작하는 기계 번역기를 구현
- 전처리 과정과 embedding 층을 사용
- 11장에서 다루었던 Tokenizer와 sequence padding을 사용

### Word-level 번역기 (영어-불어)
(사용 데이터)
- 앞서 사용했던 영-불 병렬 코퍼스를 이용
- 33,000개의 샘플을 사용

(전처리 함수)
- 문자 텍스트 데이터를 Tokenizer 함수들을 이용하여 숫자로 변환
- 영어와 불어의 각 문자을 일정 길이의 sequence로 변환

### load_preprocessed_data 함수
- 입력: 병렬 코퍼스인 fra.txt 파일을 읽음
- 출력: encoder_input(영어), decoder_input, decoder_target(불어) // 출력 데이터는 문자로 구성된 텍스트임

(수행내용)
- 텍스트에서 단어와 구두점 사이에 공백을 추가
- 알파벳과 특수 문자(".", "?", "!", ",")를 제외하고 모두 공백으로 변환
- 불어 문장에는 "<"sos">"와 "<"eos">"를 추가
- 훈련시 Teacher forcing을 사용하므로 실제 시퀀스를 저장하는 decoder_target 문장을 구성

### 정수 인코딩 과정
- 앞서 설명한 케라스 Tokenizer 함수를 이용하여 문자 텍스트를 정수 시퀀스로 변환

(수행 절차)
- fit_on_texts : 전체 데이터를 이용하여 각 단어별로 정수를 배정
- texts_to_sequences : 배정된 정수를 이용하여 문자 텍스트를 숫자 시퀀스로 변환
- pad_sequences : 각 문장을 일정한 길이로 패딩

In [ ]:
# 정수 인코딩 코드

tokenizer_en = Tokenizer(filters = "", lower = False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)
# 문자 텍스트 sents_en_in을 숫자 시퀀스인 encoder_input으로 변환

tokenizer_fra = Tokenizer(filters = "", lower = False)
tokenizer_fra.fit_on_texts(sents_fra_in)
tokenizer_fra.fit_on_texts(sents_fra_out)
decoder_input = tokenizer_fra.texts_to_sequences(sents_fra_in)
decoder_target = tokenizer_fra.texts_to_sequences(sents_fra_out)
# 문자 텍스트 sents_fra_in을 숫자 시퀀스인 decoder_input으로 변환

encoder_input = pad_sequences(encoder_input, padding = "post")
decoder_input = pad_sequences(decoder_input, padding = "post")
decoder_target = pad_sequences(decoder_target, padding = "post")
# 숫자 시퀀스 encoder_input, decoder_input을 일정 길이로 패딩.
# 이 데이터가 신경망 입력으로 사용됨.

In [ ]:
# 사용된 단어 개수
# 사용된 단어는 Tokenizer의 word_index 길이를 통해 알 수 있음

src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print("영어 단어 집합의 크기 : {:d}, 프랑스어 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

# 영어~크기: 4663, 프랑스어~크기 : 8038

### 출력 문장을 생성할 딕셔너리
- 숫자 시퀀스에서 다시 문자 텍스트를 생성하기 위해 단어에서 정수를 얻는 딕셔너리와 정수에서 단어를 얻는 딕셔너리를 저장

In [ ]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_fra.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_fra.index_word # 훈련 후 결과 비교할 때 사용

### 훈련/테스트 데이터 분리
- 33,000 샘플 중 10%인 3,300개를 테스트 데이터로 사용

In [ ]:
n_of_val = int(33000*0.1)

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

### 기계번역기 만들기
- 함수형 API를 이용하여 구현
- 임베딩 벡터 크기와 LSTM 은닉 상태의 크기를 50개로 설정
- 인코더는 아래와 같음

In [ ]:
latent_dim = 50

# 인코더
encoder_inputs = Input(shape = (None, ))
enc_emb = Embedding(src_vocab_size, latent_dim)(encoder_inputs) # 임베딩 층
enc_masking = Masking(mask_value = 0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state = True)
# 상태값 리턴을 위해 return_state는 True

encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
# 은닉 상태와 셀 상태를 리턴

encoder_states = [state_h, state_c] # 인코더의 은닉 상태와 셀 상태를 저장

- 디코더는 아래와 같음

In [ ]:
# 디코더
decoder_inputs = Input(shape = (None, ))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value = 0.0)(dec_emb)

# 상태값 리턴을 위해 return_state는 True
# 모든 시점에 대해서 단어를 예측하기 위해 return_seuences는 True
decoder_lstm = LSTM(latent_dim, return_sequences = True, return_state = True)

# 인코더의 은닉 상태를 초기 은닉 상태(initial_state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state = encoder_states)

# 모든 시점의 결과에 대해서 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

### 모델의 구조: model.summary()

### 신경망 훈련과 결과
- 모델의 생성과 훈련

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer = 'rmsprop',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['acc'])

model.fit(x = [encoder_input_train, decoder_input_train],
          y = decoder_target_train,
          validation_data = ([encoder_input_test, decoder_input_test],
                             decoder_target_test),
          batch_size = 128,
          epochs = 50)

### 기계 번역기 동작시키기
- 인코더와 디코더 정의

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape = (latent_dim, ))
decoder_state_input_c = Input(shape = (latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층을 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 디코더 정의
decoder_moel = Model([decoder_inputs] + decoder_states_inputs,
                     [decoder_outputs2] + decoder_states2)

### seq2seq 디코더 동작 코드

In [ ]:
def decode_sequence(input_seq):
    
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    
    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.
    stop_condition = False
    decoded_sentence = ""
    
    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        
        # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_toens, h, c = decoder_model.predict([target_seq] + state_value)
        
        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        
        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char
        
        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len):
            stop_condition = True
        
        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.
        
        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]
        
    return decoded_sentence

### 결과 확인을 위한 문장 생성
- 정수 시퀀스를 텍스트 시퀀스로 변환

In [ ]:
def seq2src(input_seq):
    temp = ''
    for i in input_seq:
        if(i != 0):
            temp = temp + index_to_src[i] + ' '
    return temp

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp = ''
    for i in input_seq:
        if ((i != 0 and i != tar_to_index['<sos>']) and i != tar_to_index['<eos>']):
            temp = temp + index_to_tar[i] + ' '
    return temp

### 임의의 샘플에 대한 번역 결과
- 정수 시퀀스를 텍스트 시퀀스로 변환

In [ ]:
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    
    print("원문 : ", seq2src(encoder_input_train[seq_index]))
    print("번역문 : ", seq2tar(decoder_input_train[seq_index]))
    print("예측문 : ", decoded_sentence[:-5])
    print("\n")